In [16]:
from hummingbird.ml import convert, load
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import numpy as np
import tensorflow as tf
import torch
import onnx
import onnxruntime

In [2]:
forest = RandomForestClassifier(n_estimators=105)
X, y = make_classification(n_samples=1000, n_features=8,
                           n_informative=4, n_redundant=1,
                           random_state=0, shuffle=True,
                           n_classes=4)

In [4]:
forest.fit(X, y)

RandomForestClassifier(n_estimators=105)

### Find in: Hummingbird/ml/operator_converters/_tree_implementations.py

### GEnericMatrixMultiplication

In [67]:
model = convert(forest, 'torch', X, extra_config={"tree_implementation":"gemm"})

In [68]:
torch_model = model.model

In [80]:
with torch.no_grad():
    for op in torch_model._operators:
        print(f'A: {op.weight_1} shape: {op.weight_1.shape}\n')
        print(f'B: {op.bias_1} shape: {op.bias_1.shape}\n')
        print(f'C: {op.weight_2} shape: {op.weight_2.shape}\n')
        print(f'D: {op.bias_2} shape: {op.bias_2.shape}\n')
        print(f'E: {op.weight_3} shape: {op.weight_3.shape}\n')

A: Parameter containing:
tensor([[0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True) shape: torch.Size([22680, 8])

B: Parameter containing:
tensor([[-0.2032],
        [-0.0414],
        [-1.2543],
        ...,
        [ 0.0000],
        [ 0.0000],
        [ 0.0000]], requires_grad=True) shape: torch.Size([22680, 1])

C: Parameter containing:
tensor([[[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
 

### TreeTraversal

In [ ]:
model = convert(forest, 'torch', X)

In [ ]:
torch_model = model.model

In [63]:
with torch.no_grad():
    for op in torch_model._operators:
        print('All attributes: ')
        print(op.n_features)
        print(op.max_tree_depth)
        print(op.num_trees)
        print(op.num_nodes)
        print(f'lefts: {op.lefts} length: {op.lefts.shape}\n')
        print(f'rights: {op.rights} length: {op.rights.shape}\n')
        print(f'features: {op.features} length: {op.features.shape}\n')
        print(f'thresholds: {op.thresholds}\n')
        print(f'values:  {op.values}\n')
        print(f'nodes_offset: {op.nodes_offset}')

All attributes: 
8
21
105
433
lefts: Parameter containing:
tensor([1, 2, 3,  ..., 0, 0, 0]) length: torch.Size([45465])

rights: Parameter containing:
tensor([136,  51,  10,  ...,   0,   0,   0]) length: torch.Size([45465])

features: Parameter containing:
tensor([5, 2, 0,  ..., 0, 0, 0]) length: torch.Size([45465])

thresholds: Parameter containing:
tensor([-0.2032, -0.0414, -1.2543,  ...,  0.0000,  0.0000,  0.0000],
       requires_grad=True)

values:  Parameter containing:
tensor([[0.0024, 0.0025, 0.0022, 0.0024],
        [0.0004, 0.0037, 0.0037, 0.0018],
        [0.0005, 0.0007, 0.0079, 0.0004],
        ...,
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]], requires_grad=True)

nodes_offset: Parameter containing:
tensor([[    0,   433,   866,  1299,  1732,  2165,  2598,  3031,  3464,  3897,
          4330,  4763,  5196,  5629,  6062,  6495,  6928,  7361,  7794,  8227,
          8660,  9093,  9526,  9959, 1

In [56]:
def forward(op, x):
        indexes = op._expand_indexes(x.size()[0])

        for _ in range(op.max_tree_depth):
            tree_nodes = indexes
            feature_nodes = torch.index_select(op.features, 0, tree_nodes).view(-1, op.num_trees)
            feature_values = torch.gather(x, 1, feature_nodes)

            thresholds = torch.index_select(op.thresholds, 0, indexes).view(-1, op.num_trees)
            lefts = torch.index_select(op.lefts, 0, indexes).view(-1, op.num_trees)
            rights = torch.index_select(op.rights, 0, indexes).view(-1, op.num_trees)

            indexes = torch.where(op.decision_cond(feature_values, thresholds), lefts, rights).long()
            indexes = indexes + op.nodes_offset
            indexes = indexes.view(-1)

        output = torch.index_select(op.values, 0, indexes).view(-1, op.num_trees, op.n_classes)

        output = op.aggregation(output)

        if op.regression:
            return output

        if op.anomaly_detection:
            # Select the class (-1 if negative) and return the score.
            return torch.where(output.view(-1) < 0, op.classes[0], op.classes[1]), output

        if op.perform_class_select:
            return torch.index_select(op.classes, 0, torch.argmax(output, dim=1)), output
        else:
            return torch.argmax(output, dim=1), 